In [1]:
import sys 

import numpy as np
#import wandb

##  Paths zone

In [2]:
# Watchmal inner import paths
watchmal_global = "/sps/t2k/eleblevec/updated_watchmal/WatChMaL"

In [3]:
sys.path.append(watchmal_global)

In [4]:
sys.path

['/sps/t2k/eleblevec/updated_watchmal/WatChMaL/watchmal',
 '/sps/t2k/eleblevec/miniconda3/envs/grant/lib/python310.zip',
 '/sps/t2k/eleblevec/miniconda3/envs/grant/lib/python3.10',
 '/sps/t2k/eleblevec/miniconda3/envs/grant/lib/python3.10/lib-dynload',
 '',
 '/sps/t2k/eleblevec/miniconda3/envs/grant/lib/python3.10/site-packages',
 '/sps/t2k/eleblevec/updated_watchmal/WatChMaL']

### Tests zone

In [5]:
from dataset.gnn import transformations

import dataset.data_utils as du

In [6]:
a = du.get_transformations(transformations, ['sklearnNearestNeighbors'])
a

[<function dataset.gnn.transformations.sklearnNearestNeighbors(features, n_neighbors=10)>]

In [8]:
import numpy as np
import uproot

import torch
from torch_geometric.data import InMemoryDataset, Data

# Watchmal imports
import watchmal.dataset.data_utils as du
from watchmal.dataset.gnn import transformations



# À FAIRE :
 
# Ajouter une vérification si le config n'a pas été modifié 
#    Cad enregistrer le .config dans processed/
# 30/01 Modifier le nom pre_transform en quelque chose de plus explicite.
#    Et expliquer le but des variables pre_filter, transforms, pre_transforms [les nouvelles..]


class GraphInMemoryDataset(InMemoryDataset):
    r"""" Last update if this documentation : Erwan - 25/01
    Args :
        graph_dataset_path: string
            Location of the .pt of PROCESSED graphs
        beta_mode: bool
            Indicates if the dataset is in debug mode (max verbose)
        pre_transform: function
            Function to execute to create a graph from a root event
    
    Notes :
        About the graph creation from an event : decision NOT TO USE the pre_transform
        parameter existing in the InMemoryDataset class of torch_geometric because 
            1. The name would not be explicit
            2. Is it supposed to be a list of functions to apply to the data points 
                before turning them into a graph. So it should not changes the nature 
                of the data. Or creating a graph changes the nature of the data.
            3. There should be only one function to call to create a graph, so a list
                of function is not adapted.
    """

    def __init__(
            self, 
            config=None, # Pas encore 100% utilisé (en prévision de wandb)
            pre_filter=None,
            transform=None,
            transforms=None # For compatibility with watchmal. In discussion with Nick to solve this redundancy.
    ):
    
        # General variables
        print(type(config))
        print(config)

        self.config    = config
        self.beta_mode = config["beta_mode"]
        self.verbose = 2 if self.beta_mode else config["verbose"] 

        # Variables to read a .root file
        self.root_init = False

        # Variables to create the graphs
        self.graph_init = False

        # Get the function to compute the edge_index from the point cloud
        self.compute_edge_index = du.get_transformations(transformations, config["compute_edge_index"])

        # Variables for PyG Dataset
        super().__init__( 
                        root=self.config["graph_dataset_path"], 
                        pre_filter=pre_filter,
                        pre_transform=None, # Pre_transform should not be used
                        transform=transform # torch basic transform argument should go there. 
        )

        # Check if there is data in graph_dataset_path
        self.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return self.root_file_path

    @property
    def processed_file_names(self):
        return ['data.pt']


    def root_initialize(self, config):
        """
        Method to initialize variables associated to the reading of a .root file
        Not mandatory in a InMemoryDataset
        """
        self.root_file_path = config['root_file_path']
        self.tree_name      = config['tree_name']

        self.root_init = True

    
    def graph_initialize(self, config):
        """
        Method to initialize variables associated to the creation of a graph given a .root file.
        I. e. the values uproot is going to look for.
        This method should belong to the RootDataset class (à faire)
        """
        self.train_keys   = config['train_keys']
        self.label_keys   = config['label_keys']
        self.edge_keys    = config['edge_keys']
        self.n_neighbors  = config['n_neighbors'] 

        self.graph_init = True
        


    def process(self):
        # If process() is called it means that path_to_gnn_dataset is empty
        # So the program needs to get the data from the .root file
        print('ok 0')
        print(du.get_transformations(transformations, self.config["compute_edge_index"]))
        print(self.compute_edge_index)

        if not self.root_init:
            self.root_initialize(self.config)

        if not self.graph_init:
            self.graph_initialize(self.config)

        data_list = []

        # On filtre les clés du ttree si nécessaire
        if self.pre_filter is not None:
            pass 
            # Pour l'instant ne fait rien (la fonction filtre doit retourner vrai ou faux)
            # Exemple :
            # data_list = [data for data in data_list if self.pre_filter(data)]


        # On créé les graphes à partir des données brutes du ttree
        # À FAIRE : Changer le paramètre pre_transform en 
        # un nom de paramètre plus explicite (comme compute_graph)
        # L'argument doit rester otpionnel pour conserver la possibilité de créer les graphes via les couches.
        # Dans ce cas edge_index = None (ou on ne le précise pas ?)
        if self.compute_edge_index is not None:
            print('ok 1')
            t = self.compute_edge_index[0] # Should disapear in the new versions

            with uproot.open(self.root_file_path) as root_file: # Obligé de garder le fichier ouvert pour la lecture
                root_tree = root_file[self.tree_name]

                train_data  = root_tree.arrays(self.train_keys, library='np')
                labels_data = root_tree.arrays(self.label_keys, library='np')
                edge_data   = root_tree.arrays(self.edge_keys, library='np')

                for i in range(root_tree.num_entries):
                    print('ok 2')
                    node_features = np.stack([value[i] for value in train_data.values()], axis=-1)
                    labels        = np.stack([values[i] for values in labels_data.values()], axis=-1)

                    edge_features = np.stack([value[i] for value in edge_data.values()], axis=-1)
                    edge_index    = t(features=edge_features, n_neighbors=self.n_neighbors)

                    graph = Data(x=node_features, edge_index=edge_index, y=labels)
                    data_list.append(graph)

                    if self.beta_mode and ( i == 100 ):
                        break
                    
                    if self.verbose >= 1:
                        if i % 5 == 0:
                            print(f"\nÉvènement numéro {i}")
                            print(f"edge Features : {edge_features.shape}")
                            print(f"Index : {edge_index.shape}")
                            print(f"Node features : {node_features.shape}")
                            print(f"Labels : {labels.shape}")

        self.save(data_list, self.processed_paths[0])


    def get(self, idx):
        """
        Linear layers need torch tensor as input, not ndarray

        This overcharge of the get method convert numpy arrays (the type of .x, .edge_index and .y) to torchTensors if they are not already torch tensors
        """

        # Caution : If transform functions are given in __init__(), this data object will have already be transformed
        data = super().get(idx)

        if isinstance(data.x, np.ndarray):
            data.x = torch.from_numpy(data.x).to(torch.float32) # What if float16 ?

        if isinstance(data.edge_index, np.ndarray):
            data.edge_index = torch.from_numpy(data.edge_index).to(torch.int64) # Maybe .long() ?

        if isinstance(data.y, np.ndarray):
            data.y = torch.from_numpy(data.y).to(torch.float16) # float kind mandatory for BCEloss (16, 32, ..)
            

        return data



    
# Remarque : Si on souhaite transformer les données il faut créer une fonction transform
# et la passer en paramètre au début
# (Ou surcharger la fonction get de InMemoryDataset)


In [9]:
from omegaconf import OmegaConf

yaml_content = """
config:
  root_file_path: /sps/t2k/eleblevec/files_for_new_watchmal/datasets/root_datasets/output_test.root
  tree_name: 'root_event'
  graph_dataset_path: /sps/t2k/eleblevec/updated_watchmal/WatChMaL/outputs/GraphDatasets/
  train_keys: ['hitx', 'hity', 'hitz', 'charge', 'time']
  label_keys: ['eventType']
  edge_keys: ['hitx', 'hity', 'hitz']
  n_neighbors: 10
  compute_edge_index:
    - sklearnNearestNeighbors
  beta_mode: True
  verbose: 0
"""

# Convert the YAML string to an OmegaConf DictConfig
config = OmegaConf.create(yaml_content)

# Accessing elements (DictConfig objects can be accessed like regular dictionaries)
print(config.config.root_file_path)
print(config.config.train_keys)
config


/sps/t2k/eleblevec/files_for_new_watchmal/datasets/root_datasets/output_test.root
['hitx', 'hity', 'hitz', 'charge', 'time']


{'config': {'root_file_path': '/sps/t2k/eleblevec/files_for_new_watchmal/datasets/root_datasets/output_test.root', 'tree_name': 'root_event', 'graph_dataset_path': '/sps/t2k/eleblevec/updated_watchmal/WatChMaL/outputs/GraphDatasets/', 'train_keys': ['hitx', 'hity', 'hitz', 'charge', 'time'], 'label_keys': ['eventType'], 'edge_keys': ['hitx', 'hity', 'hitz'], 'n_neighbors': 10, 'compute_edge_index': ['sklearnNearestNeighbors'], 'beta_mode': True, 'verbose': 0}}

In [10]:
dataset = GraphInMemoryDataset(config["config"])

<class 'omegaconf.dictconfig.DictConfig'>
{'root_file_path': '/sps/t2k/eleblevec/files_for_new_watchmal/datasets/root_datasets/output_test.root', 'tree_name': 'root_event', 'graph_dataset_path': '/sps/t2k/eleblevec/updated_watchmal/WatChMaL/outputs/GraphDatasets/', 'train_keys': ['hitx', 'hity', 'hitz', 'charge', 'time'], 'label_keys': ['eventType'], 'edge_keys': ['hitx', 'hity', 'hitz'], 'n_neighbors': 10, 'compute_edge_index': ['sklearnNearestNeighbors'], 'beta_mode': True, 'verbose': 0}
ok 0
[<function sklearnNearestNeighbors at 0x7f078658a5f0>]
[<function sklearnNearestNeighbors at 0x7f078658a5f0>]
ok 1


Processing...


ok 2

Évènement numéro 0
edge Features : (2431, 3)
Index : torch.Size([2, 24310])
Node features : (2431, 5)
Labels : (1,)
ok 2
ok 2
ok 2
ok 2
ok 2

Évènement numéro 5
edge Features : (3033, 3)
Index : torch.Size([2, 30330])
Node features : (3033, 5)
Labels : (1,)
ok 2
ok 2
ok 2
ok 2
ok 2

Évènement numéro 10
edge Features : (4894, 3)
Index : torch.Size([2, 48940])
Node features : (4894, 5)
Labels : (1,)
ok 2
ok 2
ok 2
ok 2
ok 2

Évènement numéro 15
edge Features : (3207, 3)
Index : torch.Size([2, 32070])
Node features : (3207, 5)
Labels : (1,)
ok 2
ok 2
ok 2
ok 2
ok 2

Évènement numéro 20
edge Features : (4169, 3)
Index : torch.Size([2, 41690])
Node features : (4169, 5)
Labels : (1,)
ok 2
ok 2
ok 2
ok 2
ok 2

Évènement numéro 25
edge Features : (4623, 3)
Index : torch.Size([2, 46230])
Node features : (4623, 5)
Labels : (1,)
ok 2
ok 2
ok 2
ok 2
ok 2

Évènement numéro 30
edge Features : (2262, 3)
Index : torch.Size([2, 22620])
Node features : (2262, 5)
Labels : (1,)
ok 2
ok 2
ok 2
ok 2
o

Done!


In [15]:
dataset.compute_edge_index

[<function watchmal.dataset.gnn.transformations.sklearnNearestNeighbors(features, n_neighbors=10)>]